# Exercise - Enable Tool Calling - STARTER

In this exercise, you’ll enhance your AI agent by adding tool-calling capabilities, allowing it to interact with external functions dynamically.

**Challenge**

Imagine you're building an AI-powered assistant that helps users with various tasks such as:

- Fetching real-time stock prices
- Performing complex calculations
- Querying a weather API
- Searching a database

Instead of manually deciding when to call which function, your AI agent will automatically detect when a tool is needed and invoke it.

## 0. Import the necessary libs

In [205]:
%pip install python-dotenv pendulum


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [206]:
import requests
import datetime
import inspect
import json
from typing import (
    TypedDict,
    List, Dict, Literal,
    Callable, Optional, Any,
    get_type_hints
)
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall
from dotenv import load_dotenv
import os

## 1. Recap: how to use OpenAI client with your API Key

To be able to connect with OpenAI, you need to instantiate an OpenAI client passing your OpenAI key.

You can pass the `api_key` argument directly.
```python
client = OpenAI(api_key="voc-")
```

In [207]:
load_dotenv("../.env")

True

In [208]:
client = OpenAI(
     api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openai.vocareum.com/v1"
    )

In [209]:
system_prompt = "Act as Senior Python Programmer. You don't know anything about other programming language, so don't provide answers about languanges like like Java."
user_question = "What is the Java Virtual Machine?"

response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_question},
        ],
        temperature=0.0,
    )
response.choices[0].message.content

"I'm focused on Python programming, so I don't have information about the Java Virtual Machine or other languages. However, if you have any questions about Python, feel free to ask!"

## 2. Recap: Memory & Function

Recently we combined memory with custom Python functions to create the full cycle of tool calling, which enabled an LLM to interact with the world.

In [210]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []

    def add_message(self,
                    role: Literal['user', 'system', 'assistant', 'tool'],
                    content: str,
                    tool_calls: dict=dict(),
                    tool_call_id=None)-> None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def reset(self) -> None:
        self._messages = []

In [211]:
def chat_with_tools(user_question:str=None,
                    memory:Memory=None,
                    model:str="gpt-4o-mini",
                    temperature=0.0,
                    tools=None)-> str:
    messages = [{"role": "user", "content": user_question}]

    if memory:
        if user_question:
            memory.add_message(role="user", content=user_question)
        messages = memory.get_messages()

    response = client.chat.completions.create(
        model = model,
        temperature = temperature,
        messages = messages,
        tools=tools,
    )

    ai_message = str(response.choices[0].message.content)
    tool_calls = response.choices[0].message.tool_calls

    if memory:
        memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)

    return ai_message

In [212]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""

    return base ** exponent

In [213]:
tools = [{
    "type": "function",
    "function": {
        "name": "power",
        "description": "Exponentatiation: base to the power of exponent",
        "parameters": {
            "type": "object",
            "properties": {
                "base": {"type": "number"},
                "exponent": {"type": "number"}
            },
            "required": ["base", "exponent"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [214]:
# Instantiate memory and start with the system prompt
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

# Call the LLM with a question that needs a tool
ai_message = chat_with_tools(
    "2 to the power of -5?",
    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

# Get the arguments from the tool_calls object and call the actual defined function
args = json.loads(memory.last_message()['tool_calls'][0].function.arguments)
result = power(args["base"], args["exponent"])

# Extract the tool_call_id and feed the LLM with the result from the function
tool_call_id = memory.last_message()['tool_calls'][0].id
memory.add_message(role="tool", content=str(result), tool_call_id=tool_call_id)
ai_message = chat_with_tools(
    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

In [215]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_K4TPChiwiQwxZ0q26SY7i4BO', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_K4TPChiwiQwxZ0q26SY7i4BO'},
 {'role': 'assistant',
  'content': '2 to the power of -5 is 0.03125.',
  'tool_calls': None}]

## 3. Create Tool abstractions

Although powerful, the way we've built by calling manually is prone to errors. What if you don't pass the correct type or miss one required field in the json-schema?

Your task is creating an abstraction to make it easier to build a tool and call it. 

Tip: Inspect the json schema of the tool we created to help you.

Your class should have at least the following methods: 
- `__init__()` receiving the function and some logic to extract docs, arguments and their types
- `dict()` to return the json schema
- `__call__()` to enable the object instantiated to be callable. 

Example:
```python
class Tool:
    def __init__(self, func:Callable):
        self.func = func
    
    def dict(self):
        pass

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)  

def my_func(arg1:int)->str:
    return "ok"

my_tool = Tool(my_func)
my_tool(arg1=1)
```



It's important learn about the following Python methods to understand how to parse functions and get docstrings, arguments and types:
- typing.get_type_hints()
- inspect.signature()

In [216]:
from typing import get_type_hints

class Tool:
    def __init__(self, func:Callable):
        self.func = func
        self.name = func.__name__
        self.args = inspect.getfullargspec(func).args
        self.type_hints = get_type_hints(func)

    def dict(self):
        properties = {}
        type_map = {
            int: "integer",
            float: "number",
            str: "string",
            bool: "boolean",
            list: "array",
            dict: "object",
        }

        for arg in self.args:
            arg_type = self.type_hints.get(arg, str)
            json_type = type_map.get(arg_type, "string")
            properties[arg] = {"type": json_type}

        return {
            "type": "function",
            "function": {
                "name": self.func.__name__,
                "description": self.func.__doc__,
                "parameters": {
                    "type": "object",
                    "properties": properties,  # ✅ NOW POPULATED
                    "required": self.args,
                    "additionalProperties": False
                },
                "strict": True
            }
        }

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [217]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""

    return base ** exponent

In [218]:
power_tool = Tool(power)

In [219]:
power_tool.dict()

{'type': 'function',
 'function': {'name': 'power',
  'description': 'Exponentatiation: base to the power of exponent',
  'parameters': {'type': 'object',
   'properties': {'base': {'type': 'number'}, 'exponent': {'type': 'number'}},
   'required': ['base', 'exponent'],
   'additionalProperties': False},
  'strict': True}}

In [220]:
power_tool(2,3)

8

## 3. Update the Agent class

You will enhance the logic of your agent so that it can handle user queries and interact with external tools dynamically. The goal is to refine how the agent processes user messages, generates responses, and invokes tools when necessary.

**Objective**

You will modify the logic responsible for:

- Processing user input – The agent should record and manage conversation history.
- Generating a response – The agent will use a language model to create a reply based on previous messages.
- Identifying when tools are needed – If a tool is required to complete the request, the agent should detect this and trigger the appropriate function.
- Handling tool execution and responses – The agent should execute the tool, capture its output, and integrate the result into the conversation.

**Steps**

- Update the logic to check whether a tool needs to be invoked based on the AI-generated response.
- If tools are needed, execute them with the correct arguments.
- Incorporate tool results into the conversation so that the AI can refine its response using the additional information.
- Ensure the agent can handle multiple tool calls recursively, meaning if the response suggests using another tool after the first one, it should be processed correctly.

**Considerations**

- Think about how the agent decides when to call a tool.
- Ensure the agent stores the tool’s response correctly so it can continue the conversation naturally.
- Handle cases where multiple tools might need to be used in sequence before the final response is given.

In [221]:
class Agent:
    """A tool-calling AI Agent"""

    def __init__(
        self,
        name:str = "Agent",
        role:str = "Personal Assistant",
        instructions:str = "Help users with any question",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
        tools:List[Tool] = [],
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"You're an AI Agent, your role is {self.role}, "
                    f"and you need to {self.instructions}",
        )

        self.client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            base_url="https://openai.vocareum.com/v1"
        )

        self.tools = tools

    def invoke(self, user_message: str) -> str:
        # Add user message to memory
        self.memory.add_message(role="user", content=user_message)


        # Convert tools to dict format
        tools_dict = [tool.dict() for tool in self.tools]

        # First API call
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=self.memory.get_messages(),
            tools=tools_dict if tools_dict else None,
        )

        ai_message = response.choices[0].message.content or ""
        tool_calls = response.choices[0].message.tool_calls

        # Store AI response
        self.memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)

        # Handle tool calls
        if tool_calls:
            for tool_call in tool_calls:
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)

                # Find and execute the matching tool
                for tool in self.tools:
                    if tool.name == tool_name:
                        try:
                            result = tool(**tool_args)
                            self.memory.add_message(
                                role="tool",
                                content=str(result),
                                tool_call_id=tool_call.id
                            )
                        except Exception as e:
                            self.memory.add_message(
                                role="tool",
                                content=f"Error: {str(e)}",
                                tool_call_id=tool_call.id
                            )
                        break

            # Ask AI again with tool results
            response = self.client.chat.completions.create(
                model=self.model,
                temperature=self.temperature,
                messages=self.memory.get_messages(),
                tools=tools_dict if tools_dict else None,
            )
            ai_message = response.choices[0].message.content or ""
            self.memory.add_message(role="assistant", content=ai_message, tool_calls=response.choices[0].message.tool_calls)

        return ai_message

## 3. Build some agents and have fun

Create some specific agents with  tools, invoke then and  inspect their memory

In [222]:
import pendulum
def get_current_time(timezone: str) -> str:
    """Get the current time in a specific timezone."""
    try:
        import pendulum
        current_time = pendulum.now(timezone)
        return current_time.format("YYYY-MM-DD HH:mm:ss ZZZ")
    except:
        return "Invalid timezone"


In [223]:
# TODO  - Create a default agent passing the tools you created
time_tool = Tool(get_current_time)
agent = Agent(name="TimeAssistant",
    role="Time helper",
    instructions="Help users get current time in timezone in which they mention in the prompt", tools=[time_tool])

In [224]:
# TODO - Ask it simple questions not related to the tools you created:
agent.invoke("What's the time now in Bucharest, Romania?")

'The current time in Bucharest, Romania is 10:03 AM on March 1, 2026.'

In [225]:
# TODO - Check its memory
agent.memory.get_messages()

[{'role': 'system',
  'content': "You're an AI Agent, your role is Time helper, and you need to Help users get current time in timezone in which they mention in the prompt",
  'tool_calls': {}},
 {'role': 'user',
  'content': "What's the time now in Bucharest, Romania?",
  'tool_calls': {}},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_St7tQCMyZqvgIifLS0uWC9G2', function=Function(arguments='{"timezone":"Europe/Bucharest"}', name='get_current_time'), type='function')]},
 {'role': 'tool',
  'content': '2026-03-01 10:03:48 +0200+02:00',
  'tool_call_id': 'call_St7tQCMyZqvgIifLS0uWC9G2'},
 {'role': 'assistant',
  'content': 'The current time in Bucharest, Romania is 10:03 AM on March 1, 2026.',
  'tool_calls': None}]

In [226]:
# TODO - Reset your agent's memory
agent.memory.reset()

In [227]:
# TODO - Ask it simple questions related to the tools you created:
agent.invoke("What is the time now in Paris, France?")

'The current time in Paris, France is 09:03 AM on March 1, 2026.'

In [228]:
# TODO - Check its memory again
agent.memory.get_messages()

[{'role': 'user',
  'content': 'What is the time now in Paris, France?',
  'tool_calls': {}},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_gglbBTY2oK3pdxRvY382aKBS', function=Function(arguments='{"timezone":"Europe/Paris"}', name='get_current_time'), type='function')]},
 {'role': 'tool',
  'content': '2026-03-01 09:03:50 +0100+01:00',
  'tool_call_id': 'call_gglbBTY2oK3pdxRvY382aKBS'},
 {'role': 'assistant',
  'content': 'The current time in Paris, France is 09:03 AM on March 1, 2026.',
  'tool_calls': None}]

In [229]:
# TODO - Ask it simple questions related to the tools you created:
agent.invoke("What is the time now in Warsaw?")

'The current time in Warsaw, Poland is 09:03 AM on March 1, 2026.'

In [230]:
# TODO - Check its memory
agent.memory.get_messages()

[{'role': 'user',
  'content': 'What is the time now in Paris, France?',
  'tool_calls': {}},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_gglbBTY2oK3pdxRvY382aKBS', function=Function(arguments='{"timezone":"Europe/Paris"}', name='get_current_time'), type='function')]},
 {'role': 'tool',
  'content': '2026-03-01 09:03:50 +0100+01:00',
  'tool_call_id': 'call_gglbBTY2oK3pdxRvY382aKBS'},
 {'role': 'assistant',
  'content': 'The current time in Paris, France is 09:03 AM on March 1, 2026.',
  'tool_calls': None},
 {'role': 'user',
  'content': 'What is the time now in Warsaw?',
  'tool_calls': {}},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_ftOcGV3z9lZoShH7QytClURF', function=Function(arguments='{"timezone":"Europe/Warsaw"}', name='get_current_time'), type='function')]},
 {'role': 'tool',
  'content': '2026-03-01 09:03:53 +0100+01:00',
  'tool_call_id': 'call_ftOcGV3z9lZo

## 4. Experiment

Now that you understood how it works, experiment with new things.

- Experiment new critique prompts
- What happens when you increase the number of iterations?
- Try accessing the memory to inspect it (agent.memory) instead of reading the outputs (verbose=False)
- What else can you try?